**ИСУ: 409146**

In [1]:
46 % 36

10

**YAML -> XML**

# Обязательное задание

In [22]:
class Parser:
    def __init__(self, filename):
        with open(filename, 'r') as f:
            self.file = f.read()
        self.final = ''
    
    def get_as_plain_text(self):
        return self.file
    
    def render(self):
        pass
    
    def write(self, filename):
        with open(filename, 'w') as f:
            f.write(self.final)

In [23]:
class stupid_YAML_XML_Parser(Parser):
    def __init__(self, filename):
        super().__init__(filename)
        self.tags = []
        self.tag_tabs = []
        self.final = '<?xml version="1.0" encoding="UTF-8" ?>\n'
    def render(self):
        last_tabs = 0
        for string in self.file.split('\n'):
            key, value = string.rstrip().split(':', 1)
            tabs = key.count(' ') // 2
            key = key.strip()
            value = value.strip()
            self.final += f'{" "*tabs}<{key}>{value}</{key}>\n'

In [24]:
parser = stupid_YAML_XML_Parser('schedule.yaml')
parser.render()
parser.write('stupid_schedule.xml')

# Дополнительное задание 1

In [27]:
import yaml
from dict2xml import dict2xml

In [28]:
with open('schedule.yaml', 'r') as f:
    prime_service = yaml.safe_load(f)
    
xml = dict2xml(prime_service)

with open('schedule.xml', 'w') as f:
    f.write(xml)

# Дополнительное задание 2

In [7]:
import re

In [49]:
class stupid_re_YAML_XML_Parser(Parser):
    def __init__(self, filename):
        super().__init__(filename)
        self.tags = []
        self.tag_tabs = []
        self.final = '<?xml version="1.0" encoding="UTF-8" ?>\n'
    
    def render(self):
        last_tabs = 0
        for string in re.split('\n', self.file): # re
            key, value = re.split(': |:$', string.rstrip()) # re
            tabs = len(re.match(r'^\s{0,20}', key)[0]) // 2 # re
            key = key.strip()
            value = value.strip()
            self.final += f'{" "*tabs}<{key}>{value}</{key}>\n'

In [50]:
parser = stupid_re_YAML_XML_Parser('schedule.yaml')
# print(parser.get_as_plain_text())
parser.render()
parser.write('schedule.xml')

## Парсер

In [10]:
class re_YAML_XML_Parser(Parser):
    def __init__(self, filename):
        super().__init__(filename)
        self.tags = []
        self.tag_tabs = []
        self.final = '<?xml version="1.0" encoding="UTF-8" ?>\n'
    
    def __format_list(self, key, value, tabs):
        string = ''
        values = re.split(', ',value[1:-1]) # re
        for value in values:
            string += f'{" "*tabs}<{key}>{value}</{key}>\n'
        return string
    
    def __format_field(self, key, value, tabs):
        if '[' in value and ']' in value:
            return self.__format_list(key, value, tabs)
        return f'{" "*tabs}<{key}>{value}</{key}>\n'
    
    def __format_open(self, key, tabs):
        self.tags += [key]
        self.tag_tabs += [tabs]
        return f'{" "*tabs}<{key}>\n'
    
    def __format_close(self, tabs, last_tabs):
        string = ''
        while tabs < last_tabs:
            last_tag = self.tags.pop()
            last_tabs = self.tag_tabs.pop()
            string += f'{" "*last_tabs}<\{last_tag}>\n'
        return string
    
    def __format_string(self):
        key, value = re.split(': |:$', string.rstrip()) # re
    
    def render(self):
        last_tabs = 0
        for string in re.split('\n', self.file): # re
            key, value = re.split(': |:$', string.rstrip()) # re
            tabs = len(re.match(r'^\s{0,20}', key)[0]) // 2 # re
            key = key.strip()
            value = value.strip()
            
            self.final += self.__format_close(tabs, last_tabs)
            if value != '':
                self.final += self.__format_field(key, value, tabs)
            else:
                self.final += self.__format_open(key, tabs)
            last_tabs = tabs
        
        self.final += self.__format_close(0, last_tabs)

In [11]:
parser = re_YAML_XML_Parser('schedule.yaml')
# print(parser.get_as_plain_text())
parser.render()
parser.write('schedule.xml')

**программы работают идентично**

# Дополнительное задание 3

In [51]:
class YAML_XML_Parser(Parser):
    def __init__(self, filename):
        super().__init__(filename)
        self.tags = []
        self.tag_tabs = []
        self.final = '<?xml version="1.0" encoding="UTF-8" ?>\n'
    
    def __format_list(self, key, value, tabs):
        string = ''
        values = value[1:-1].split(', ')
        for value in values:
            string += f'{" "*tabs}<{key}>{value}</{key}>\n'
        return string
    
    def __format_field(self, key, value, tabs):
        if '[' in value and ']' in value:
            return self.__format_list(key, value, tabs)
        return f'{" "*tabs}<{key}>{value}</{key}>\n'
    
    def __format_open(self, key, tabs):
        self.tags += [key]
        self.tag_tabs += [tabs]
        return f'{" "*tabs}<{key}>\n'
    
    def __format_close(self, tabs, last_tabs):
        string = ''
        while tabs < last_tabs:
            last_tag = self.tags.pop()
            last_tabs = self.tag_tabs.pop()
            string += f'{" "*last_tabs}</{last_tag}>\n'
        return string
    
    def __format_el(self, tabs, key, value):
        return f'{" "*tabs}<{key}>{value}</{key}>\n'
    
    def __format_string(self):
        key, value = string.rstrip().split(':', 1)
    
    def render(self):
        last_tabs = 0
        for string in self.file.split('\n'):
            if ':' not in string:
                tabs = string.rstrip().count(' ') // 2
                value = string.strip()[2:]
                self.final += self.__format_el(tabs, key, value)
            else:
                key, value = string.rstrip().split(':', 1)
                tabs = key.count(' ') // 2
                key = key.strip()
                value = value.strip()

                self.final += self.__format_close(tabs, last_tabs)
                if value != '':
                    self.final += self.__format_field(key, value, tabs)
                else:
                    self.final += self.__format_open(key, tabs)
                last_tabs = tabs
        
        self.final += self.__format_close(0, last_tabs)

In [52]:
parser = YAML_XML_Parser('schedule.yaml')
parser.render()
parser.write('schedule.xml')

# Дополнительное задание 4

In [53]:
import time

In [54]:
def stupid_re_parser():
    parser = stupid_re_YAML_XML_Parser('schedule.yaml')
    parser.render()
    parser.write('schedule.xml')
def base_parser():
    parser = YAML_XML_Parser('schedule.yaml')
    parser.render()
    parser.write('schedule.xml')
def stupid_parser():
    parser = stupid_YAML_XML_Parser('schedule.yaml')
    parser.render()
    parser.write('schedule.xml')
def bib_parser():
    with open('schedule.yaml', 'r') as f:
        prime_service = yaml.safe_load(f)
    xml = dict2xml(prime_service)
    with open('schedule.xml', 'w') as f:
        f.write(xml)

In [55]:
start_time = time.time()
for i in range(1000):
    stupid_parser()
print('Простейший парсер: ', round(time.time() - start_time, 5))

Простейший парсер:  0.14493


In [56]:
start_time = time.time()
for i in range(1000):
    bib_parser()
print('Дополнительное задание 1(библиотеки): ', round(time.time() - start_time, 5))

Дополнительное задание 1(библиотеки):  4.24789


In [57]:
start_time = time.time()
for i in range(1000):
    stupid_re_parser()
print('Дополнительное задание 2(регулярные выражения): ', round(time.time() - start_time, 5))

Дополнительное задание 2(регулярные выражения):  0.16539


In [58]:
start_time = time.time()
for i in range(1000):
    base_parser()
print('Дополнительное задание 3(настоящий парсер): ', round(time.time() - start_time, 5))

Дополнительное задание 3(настоящий парсер):  0.1994


# Дополнительное задание 5 YAML -> MD

In [59]:
class YAML_MD_Parser(Parser):
    def __init__(self, filename):
        super().__init__(filename)
        self.tags = []
        self.tag_tabs = []
        self.final = ''
    
    def __format_list(self, key, value, tabs):
        string = f'{"#"*(tabs + 1)} {key}\n'
        values = value[1:-1].split(', ')
        for value in values:
            string += f'- {value}\n'
        return string
    
    def __format_field(self, key, value, tabs):
        # if '[' in value and ']' in value:
        #     return self.__format_list(key, value, tabs)
        return f'\t{key}: {value}\n'
    
    def __format_open(self, key, tabs):
        self.tags += [key]
        self.tag_tabs += [tabs]
        if key not in ['time', 'place', 'lecturer', 'datetime']:
            return f'{"#"*(tabs + 1)} {key}\n'
        return ''

    def __format_string(self):
        key, value = string.rstrip().split(':', 1)
    
    def render(self):
        last_tabs = 0
        for string in self.file.split('\n'):
            key, value = string.rstrip().split(':', 1)
            tabs = key.count(' ') // 4
            key = key.strip()
            value = value.strip()
            
            if value != '':
                self.final += self.__format_field(key, value, tabs)
            else:
                self.final += self.__format_open(key, tabs)
            last_tabs = tabs

In [60]:
parser = YAML_MD_Parser('schedule.yaml')
# print(parser.get_as_plain_text())
parser.render()
parser.write('schedule.md')

# schedule
## tuesday
### first
	name: Иностранный язык
	type: Практика
	weeks: [1, 2, 3, 4, 5, 6, 7, 8, 9]
	start: "8:20"
	end: "9:50"
	visit: Очно
	address: ул.Ломоносова, д.9, лит. Е
	audience: Ауд. 3212
	surname: Бревнова
	name: Татьяна
	lastname: Сергеевна
### second
	name: Иностранный язык
	type: Практика
	weeks: [1, 2, 3, 4, 5, 6, 7, 8, 9]
	start: "10:00"
	end: "11:30"
	visit: Очно
	address: ул.Ломоносова, д.9, лит. Е
	audience: Ауд. 3212
	surname: Бревнова
	name: Татьяна
	lastname: Сергеевна
### third
	name: Основы дискретной математики
	type: Лекция
	weeks: [1, 2, 3, 4, 5, 6, 7, 8, 9]
	start: "17:00"
	end: "18:30"
	visit: Очно
	address: Кронверкский пр., д.49, лит.А
	audience: Ауд. Orange Classroom (1229)
	surname: Карпов
	name: Дмитрий
	lastname: Валерьевич
### fourth
	name: Основы дискретной математики
	type: Практика
	weeks: [1, 2, 3, 4, 5, 6, 7, 8, 9]
	start: "18:40"
	end: "20:10"
	visit: Очно
	address: Кронверкский пр., д.49, лит.А
	audience: Ауд. 2316
	surname: Карпов
	name: Дмитрий
	lastname: Валерьевич
## wednesday
### first
	name: Программирование
	type: Лекция
	weeks: [1, 3, 5, 7, 9]
	start: "10:00"
	end: "11:30"
	visit: Очно
	address: ул.Ломоносова, д.9, лит. М
	audience: Ауд. Актовый зал (1216/0 (усл))
	surname: Письмак
	name: Алексей
	lastname: Евгеньевич
### second
	name: Основы профессиональной деятельности
	type: Практика
	weeks: [1, 3, 5, 7, 9]
	start: "15:20"
	end: "16:50"
	visit: Очно
	address: Кронверкский пр., д.49, лит.А
	audience: Ауд. 1327
	surname: Остапенко
	name: О.
	lastname: Д.
